# Projeto 2 de Ciência dos Dados

Integrantes: Ana Carolina Pela Tofetti, Eduardo Lima Pinelli e Rafael Seicali Malcervelli

# Introdução

O universo das apostas sempre envolveu o mundo dos esportes, seja em corridas de cavalo, jogos de futebol ou lutas de boxe. No entanto, os "e-sports" ou jogos eletrônicos - videogames - vem ganhando força em diversas competições grandiosas ja realizadas; os jogos que apresentam um cenário competitivo e profissional são muitos, como: League of Legends, Starcraft, Mortal Kombat, Rainbow Six Siege, Counter Strike Global Offensive. 

Em nosso trabalho, optamos pelo jogo Counter Strike Global Offensive como ferramenta de estudo. Queremos facilitar a escolha entre dois times que irão se enfrentar em algum campeonato futuro para que assim possamos ter mais certeza em qual deles apostar.

In [5]:
#Carregando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import seaborn as sns
import statsmodels.api as sm
from collections import Counter

In [6]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\rafae\Desktop\cdados_p2\cdados_projeto2


In [59]:
#Lendo o dataset
#Criando um novo dataframe somente com as partidas, mapas e times, excluindo as estatisticas de cada jogador, pois
#existem alguns jogadores que atualmente não estão mais participando do cenário competitivo do jogo.
data = pd.read_csv("matches_info.csv").iloc[:, 0:23]

#Apagando colunas desnecessárias
del data["Unnamed: 0"]
del data["date_match"]
del data["event_name"]

#Esta função faz com que todas as colunas sejam expostas pelo DataFrame.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

#Função que conta a quantidade de aparições dos mapas, retornando um dicionário com cada um deles e suas frequencias
def contagem(lista):
    dicionario_contagem = {}
    for i in lista:
        dicionario_contagem[i] = lista.count(i)
        
    return dicionario_contagem

#Função que recebe uma lista e retorna o item mais frequente dela.
def mais_frequente(lista): 
    contagem = Counter(lista) 
    
    return contagem.most_common(1)[0][0] 

# Selecionando algumas variáveis relevantes

In [8]:
#Criando uma lista que contenha o nome de todos os times
lista_times1 = list(data.team1)
lista_times2 = list(data.team2)
lista_times = lista_times1+lista_times2

#Criando uma lista que contenha o nome de todos os mapas
lista_mapas = []
lista_frq_mapas = []
lista_crua = list(data.map1_played) + list(data.map2_played) + list(data.map3_played)

for i in lista_crua:
    if i in lista_mapas or i == "NotPlayed":
        pass
    else:
        lista_mapas.append(i)
        
#Criando uma lista para medir a frequencia de jogos nos mapas
for i in lista_crua:
    if i == "NotPlayed":
        pass
    else:
        lista_frq_mapas.append(i)

#Criando um dicionário que mostra a frequencia que cada mapa é jogado
soma_mapas = sum(contagem(lista_frq_mapas).values())
dicionario_frequencias_mapas = {}
for k, v in contagem(lista_frq_mapas).items():
    frq = v*(100) / soma_mapas
    dicionario_frequencias_mapas[k] = frq

In [9]:
#Criando um dicionário que mostra a frequencia dos times em relação as partidas jogadas
soma_times = sum(contagem(lista_times).values())
dicionario_frequencias_times = {}
for k, v in contagem(lista_times).items():
    frq = v*(100) / soma_times
    dicionario_frequencias_times[k] = frq

#Criando um dicionário que mostra a porcentagem de vitória dos times a partir da soma de todas 
#as partidas computadas.
lista_mapa2 = []
lista_mapa3 = []

for i in data.team_winner_map2:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa2.append(i)

for i in data.team_winner_map3:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa3.append(i)

vencedores = list(data.team_winner_map1) + lista_mapa2 + lista_mapa3
soma_vencedores = sum(contagem(vencedores).values())
dicionario_frequencias_vencedores = {}
dicionario_vencedores_sorted = {}
dicionario_times_sorted = {}
dicionario_vencedores_times = {}
for k, v in contagem(vencedores).items():
    frq = v*(100) / soma_vencedores
    dicionario_frequencias_vencedores[k] = frq

#Transformando todos as keys em lowercase, para poder ordená-las melhor.
dict_lower_vencedores = {}
for k, v in dicionario_frequencias_vencedores.items():
    key = k.lower()
    dict_lower_vencedores[key] = dicionario_frequencias_vencedores[k]
    
dict_lower_times = {}
for k, v in dicionario_frequencias_times.items():
    key = k.lower()
    dict_lower_times[key] = dicionario_frequencias_times[k]
    
dict_lower_mapas = {}
for k, v in dicionario_frequencias_mapas.items():
    key = k.lower()
    dict_lower_mapas[key] = dicionario_frequencias_mapas[k]   
    
#Organizando os dicionários em ordem alfabetica, facilitando sua visualização
dict_lower_vencedores = dict((k, v) for k, v in dict_lower_vencedores.items()) 
for key in sorted(dict_lower_vencedores):
    dicionario_vencedores_sorted[key] = dict_lower_vencedores[key]
    
dict_lower_times = dict((k, v) for k, v in dict_lower_times.items()) 
for key in sorted(dict_lower_times):
    dicionario_times_sorted[key] = dict_lower_times[key]

In [10]:
#Criando um dicionário que consiste em ter como chave os times e como valor, um dicionário de mapas, contendo
#em sua chave os mapas e no valor as porcentagens de vitória que o time teve em cada mapa
dicionario_vencedores_mapas = {}
for k, v in dicionario_times_sorted.items():
    dicionario_vencedores_mapas[k] = dicionario_frequencias_mapas

#Criando um dicionário que consiste em ter como chave os times, e como valor, um dicionário dos times, contendo
#em sua chave os times e no valor as porcentagens de vitória que o time teve com cada um dos times.
for time, v in dicionario_times_sorted.items():
    dicionario_vencedores_times[time] = dicionario_times_sorted
    if time in dicionario_times_sorted:
        dicionario_copia = dicionario_times_sorted.copy()
        dicionario_copia.pop(time)
        dicionario_vencedores_times[time] = dicionario_copia
    

In [122]:
#Criando um dataset mais limpo, para facilitar na visualização com apenas as variáveis que nos importam.
estatisticas = data.drop(columns=["maps_info", "team_lost_score",
                                 "team_won_score", "result_map1_played1", 
                                "result_half_score_map1", "result_map1_played2",
                                "result_map2_played1", 
                                "result_half_score_map2", "result_map2_played2",
                                "result_map3_played1", 
                                "result_half_score_map1"])

In [118]:
#Criando uma lista de listas: cada lista representa uma linha do dataframe, ou seja, um confronto.
lista_vencedores = []
lista_adicional = []
for i in range(0, 239):
    lista_adicional.append(estatisticas.iloc[ i , 3])
    lista_adicional.append(estatisticas.iloc[ i , 6])
    lista_adicional.append(estatisticas.iloc[ i , 9])
    lista_vencedores.append(lista_adicional)
    lista_adicional = []

In [124]:
#Criando uma nova coluna "team_winner", que apresenta apenas os vencedores dos confrontos.
estatisticas["team_winner"] = "Teste"
contador = 0

#Criando uma lista que contenha apenas os times que ganharam os confrontos. Como cada confronto varia 
#seu tipo por campeonato (pode ser um jogo só ou uma melhor de 3)
lista_win = []
for lista in lista_vencedores:
    lista_win.append(mais_frequente(lista))

for mapa in estatisticas["map2_played"]:
    #Se o mapa 2 representar um "NotPlayed", significa que o confronto é de um jogo apenas, 
    #então, o vencedor do primeiro jogo é o vencedor do confronto
    if mapa == "NotPlayed":
        estatisticas["team_winner"][contador] = estatisticas["team_winner_map1"][contador]
        contador += 1
        
    else:
        estatisticas["team_winner"][contador] = lista_win[contador]
        contador += 1

In [142]:
estatisticas["confronting"] = "Teste"


for i in range(0, 240):
    estatisticas["confronting"] = lista_times1[i] + lista_times2[i]

240

In [140]:
estatisticas

,team1,team2,map1_played,team_winner_map1,team_loser_map1,map2_played,team_winner_map2,team_loser_map2,map3_played,team_winner_map3,team_winner,confrontation,confronting
0,Astralis,Virtus.pro,Nuke,Virtus.pro,Astralis,Overpass,Astralis,Virtus.pro,Train,Astralis,Astralis,AstralisVirtus.pro,ENCEAVANGAR
1,Virtus.pro,SK,Train,Virtus.pro,SK,Cobblestone,Virtus.pro,SK,Overpass,NotPlayed,Virtus.pro,Virtus.proSK,ENCEAVANGAR
2,Astralis,fnatic,Cache,Astralis,fnatic,Nuke,Astralis,fnatic,Dust2,NotPlayed,Astralis,Astralisfnatic,ENCEAVANGAR
3,FaZe,SK,Mirage,FaZe,SK,Train,SK,FaZe,Overpass,SK,SK,FaZeSK,ENCEAVANGAR
4,Virtus.pro,North,Overpass,Virtus.pro,North,Cache,North,Virtus.pro,Cobblestone,Virtus.pro,Virtus.pro,Virtus.proNorth,ENCEAVANGAR
5,fnatic,Gambit,Cache,fnatic,Gambit,Overpass,Gambit,fnatic,Dust2,fnatic,fnatic,fnaticGambit,ENCEAVANGAR
6,Natus Vincere,Astralis,Overpass,Astralis,Natus Vincere,Mirage,Natus Vincere,Astralis,Dust2,Astralis,Astralis,Natus VincereAstralis,ENCEAVANGAR
7,Liquid,Astralis,Mirage,Astralis,Liquid,NotPlayed,NotPlayed,NotPlayed,NotPlayed,NotPlayed,Astralis,LiquidAstralis,ENCEAVANGAR
8,FaZe,Envy,Nuke,FaZe,Envy,NotPlayed,NotPlayed,NotPlayed,NotPlayed,NotPlayed,FaZe,FaZeEnvy,ENCEAVANGAR
9,North,GODSENT,Overpass,North,GODSENT,NotPlayed,NotPlayed,NotPlayed,NotPlayed,NotPlayed,North,NorthGODSENT,ENCEAVANGAR
